<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In LangGraph, the Searcher is a standalone module. If the Searcher fails, the Planner and Writer don't care—they just wait for the state to update.**

**Phase 1: The Setup**

We need to install langgraph and the official LangChain integration for Gemini (langchain-google-genai)

In [ ]:
!pip install -q -U langgraph langchain langchain-google-genai tavily-python pypdf

**Phase 2: Config & State Definition**

Here we define the "Shared Memory" (State) that will be passed between agents.

In [ ]:
import os
from google.colab import userdata
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# LangChain Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from tavily import TavilyClient

# 1. Setup API Keys
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY') # Note: Ensure this matches your Secret name
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except:
    print("⚠️ Error: Please set GOOGLE_API_KEY and TAVILY in Colab Secrets.")

# 2. Initialize the LLM
# We use 'gemini-1.5-flash' as it is fast and efficient for agent workflows
llm = ChatGoogleGenerativeAI(model="gemini-pro-latest", temperature=0)

# 3. Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

print("Setup complete.")

Setup complete.


**DEFINE THE STATE (The "Clipboard"):**

 This is the data structure passed between agents

In [ ]:
# 4. DEFINE THE STATE (The "Clipboard")
# This is the data structure passed between agents
class AgentState(TypedDict):
    topic: str                # The User's input (or PDF content)
    research_plan: List[str]  # Output from Planner
    search_results: str       # Output from Searcher
    final_report: str         # Output from Writer

# **Memory System**

In [ ]:
import os
import json
import datetime

# --- MEMORY SYSTEM ---
class HistoryManager:
    def __init__(self):
        self.history_file = "agent_history.json"
        self.history = self.load_history()

    def load_history(self):
        """Loads history from a JSON file if it exists."""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as f:
                return json.load(f)
        return []

    def save_entry(self, input_text, mode, final_report):
        """Saves a new research session to memory."""
        entry = {
            "id": len(self.history) + 1,
            "timestamp": str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),
            "mode": mode,
            "input": input_text[:50] + "..." if len(input_text) > 50 else input_text, # Preview only
            "full_input": input_text,
            "report": final_report
        }
        self.history.append(entry)
        with open(self.history_file, 'w') as f:
            json.dump(self.history, f)
        print("✅ Research saved to History.")

    def show_history(self):
        """Displays list of past inputs."""
        if not self.history:
            print("\n📭 History is empty.")
            return None

        print("\n📚 RESEARCH HISTORY:")
        print(f"{'ID':<4} | {'Time':<18} | {'Mode':<8} | {'Topic/Input'}")
        print("-" * 60)
        for item in self.history:
            print(f"{item['id']:<4} | {item['timestamp']:<18} | {item['mode']:<8} | {item['input']}")
        return self.history
# Initialize Memory
memory = HistoryManager()

**Phase 3: The Agents (Nodes)**

We will define your three agents as specific functions. Each function receives the state, does work, and returns an update to the state.

**1. The Planner Agent**

Breaks the topic down.

In [ ]:
def planner_node(state: AgentState):
    print(f"🧠 [Planner]: Analying request...")
    topic = state['topic']

    # We ask the LLM to return a clean list
    prompt = f"""
    You are a Research Planner.
    Topic/Context: {topic}

    Task: Generate 3 distinct, specific search queries to gather comprehensive information.
    Constraint: Return ONLY the 3 queries separated by newlines. Do not number them.
    """

    response = llm.invoke(prompt)

    # Process output into a list
    queries = [q.strip() for q in response.content.split('\n') if q.strip()]
    print(f"   -> Generated Plan: {queries[:3]}")

    return {"research_plan": queries[:3]}

**2. The Searcher Agent**

Executes the plan.

In [ ]:
def searcher_node(state: AgentState):
    print(f"🔎 [Searcher]: Executing research plan...")
    queries = state['research_plan']
    results = []

    for q in queries:
        print(f"   - Searching: {q}")
        try:
            # search_depth="advanced" gives better quality for reports
            response = tavily.search(query=q, max_results=1, search_depth="basic")
            if response['results']:
                content = response['results'][0]['content']
                results.append(f"Source: {q}\nContent: {content}")
            else:
                results.append(f"Source: {q}\nContent: No data found.")
        except Exception as e:
            print(f"Error searching {q}: {e}")

    # Combine all results into one string
    combined_content = "\n\n".join(results)
    return {"search_results": combined_content}

**3. The Writer Agent**

Synthesizes the report

In [ ]:
def writer_node(state: AgentState):
    print(f"✍️ [Writer]: Synthesizing final report...")

    topic = state['topic']
    data = state['search_results']

    prompt = f"""
    You are a Senior Technical Writer.

    Original Topic/Context: {topic[:500]}... (truncated for brevity)

    Verified Research Data:
    {data}

    Task: Write a professional, structured summary.
    1. Introduction
    2. Key Findings (incorporate the research data)
    3. Conclusion

    Format: Markdown.
    """

    response = llm.invoke(prompt)
    return {"final_report": response.content}

**Phase 4: The Execution Pipeline (Graph)**

This is where we wire the agents together using LangGraph.

In [ ]:
# 1. Initialize Graph
workflow = StateGraph(AgentState)

# 2. Add Nodes
workflow.add_node("planner", planner_node)
workflow.add_node("searcher", searcher_node)
workflow.add_node("writer", writer_node)

# 3. Define Edges (The Flow)
workflow.set_entry_point("planner")       # User Input -> Planner
workflow.add_edge("planner", "searcher")  # Planner -> Searcher
workflow.add_edge("searcher", "writer")   # Searcher -> Writer
workflow.add_edge("writer", END)          # Writer -> Finish

# 4. Compile
app = workflow.compile()
print("✅ Multi-Agent System Compiled successfully.")

✅ Multi-Agent System Compiled successfully.


**Phase 5: Main Execution Loop (Integration)**

We will keep your existing PDF and History logic, but now we feed the data into the Graph (app.invoke) instead of calling functions manually.

In [44]:
import io
import pypdf
from google.colab import files
from IPython.display import display, Markdown

# --- HELPER: PDF EXTRACTOR ---
def extract_pdf_text(uploaded_file):
    pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file))
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() + "\n"
    return text

# --- MAIN APP ---
def main():
    print("🤖 LANGGRAPH RESEARCH AGENT INITIALIZED")

    while True:
        print("\n" + "="*15 + " STRICT HYBRID AGENT " + "="*15)
        print("1. Research a Topic (Text 💬)")
        print("2. Research a Paper (Upload PDF 📖)")
        print("3. View History 📜")
        print("4. Exit 📤")

        choice = input("Select Option: ")

        if choice == '4':
            print("Goodbye.👋")
            break

        # --- OPTION 3: HISTORY ---
        if choice == '3':
            history_data = memory.show_history()
            if history_data:
                sub_choice = input("\nEnter ID to view full report (or Press Enter to go back): ")
                if sub_choice.isdigit():
                    idx = int(sub_choice) - 1
                    if 0 <= idx < len(history_data):
                        entry = history_data[idx]
                        print(f"\n🔁 RECALLING REPORT FOR: {entry['input']}")
                        print("-" * 50)
                        display(Markdown(entry['report']))
                        input("\nPress Enter to continue...")
            continue # Skip the rest and go back to menu

        # SETUP INPUT
        mode = ""
        input_data = ""
        label = "" # Used for history title

        # Prepare Input
        user_input = ""

        if choice == '1':
            user_input = input("Enter Topic: ")
            mode = "Text"
            label = user_input
        elif choice == '2':
            print("Upload PDF:")
            uploaded = files.upload()
            if uploaded:
                fn = next(iter(uploaded))
                raw_text = extract_pdf_text(uploaded[fn])
                # We wrap the PDF text in a prompt so the Planner knows what to do
                user_input = f"Analyze and validate this paper content: {raw_text[:10000]}"
                mode = "PDF"
                label = fn
            else:
                print("No file.")
                continue
        else:
            print("Invalid option. Please try again.")
            continue

        if user_input:
            print("\n🚀 STARTING AGENT PIPELINE...\n")

            # --- THE MAGIC HAPPENS HERE ---
            # We invoke the LangGraph application
            final_state = app.invoke({"topic": user_input})

            print("\n" + "="*50)
            print("📊 FINAL REPORT")
            print("="*50 + "\n")
            display(Markdown(final_state['final_report']))
            memory.save_entry(label, mode, final_state['final_report'])

if __name__ == "__main__":
    main()

🤖 LANGGRAPH RESEARCH AGENT INITIALIZED

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 2
Upload PDF:


Saving 2511.17546v1.pdf to 2511.17546v1 (2).pdf

🚀 STARTING AGENT PIPELINE...

🧠 [Planner]: Analying request...
   -> Generated Plan: ['performance and energy resolution of cryogenic microcalorimeters for 10-40 keV x-ray spectroscopy', 'theoretical calculations of hyperfine splitting and nuclear polarization in muonic atoms Z=23-30', 'accuracy of ab initio electric field gradient (EFG) calculations for open d-shell atoms like copper and cobalt']
🔎 [Searcher]: Executing research plan...
   - Searching: performance and energy resolution of cryogenic microcalorimeters for 10-40 keV x-ray spectroscopy
   - Searching: theoretical calculations of hyperfine splitting and nuclear polarization in muonic atoms Z=23-30
   - Searching: accuracy of ab initio electric field gradient (EFG) calculations for open d-shell atoms like copper and cobalt
✍️ [Writer]: Synthesizing final report...

📊 FINAL REPORT



Of course. Here is a professional, structured summary based on the provided information.

***

### **Technical Summary: Determining Nuclear Quadrupole Moments via Muonic Atom Spectroscopy**

**Paper:** "Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms" by S. Rathi et al.

---

### 1. Introduction

The paper by S. Rathi et al. presents a methodology for the high-precision determination of nuclear electric quadrupole moments for transition elements. The approach leverages the unique properties of muonic atoms, where a muon replaces an electron. Due to the muon's significantly larger mass, it orbits much closer to the nucleus, making its energy levels exceptionally sensitive to nuclear structure, size, and shape. By precisely measuring the Lamb shift—the energy difference between specific atomic states—the authors extract reference values for the nuclear quadrupole moment, a fundamental parameter describing the deviation of the nuclear charge distribution from a perfect sphere.

### 2. Key Findings

The study's conclusions are substantiated by a combination of advanced experimental techniques and robust theoretical calculations, which are validated against existing research in the field.

*   **Advanced Experimental Technique for X-ray Spectroscopy**
    The determination of the Lamb shift relies on measuring the energy of X-rays emitted as the muon cascades down through atomic energy levels. The required precision necessitates state-of-the-art detection technology. This is supported by research into **high-resolution x-ray energy-dispersive spectrometers (EDS) based on cryogenic microcalorimeters**. These detectors provide the exceptional energy resolution needed to resolve subtle energy shifts in the 10-40 keV X-ray range, which is characteristic of muonic atom transitions. This technology is fundamental to achieving the experimental accuracy reported in the paper.

*   **Robust Theoretical Framework and Validation**
    Extracting the quadrupole moment from the measured Lamb shift requires a comprehensive theoretical model that accounts for all significant quantum electrodynamics (QED), nuclear polarization, and relativistic effects. The validity of these complex models is often benchmarked against other measurable nuclear-atomic interactions. This is consistent with **theoretical calculations of hyperfine splitting in muonic atoms (Z=23-30)**. The availability of both theoretical and experimental data on hyperfine splitting provides a crucial cross-check, ensuring the underlying models used to interpret the Lamb shift are accurate and reliable for this specific range of transition elements.

*   **Addressing Computational Challenges in EFG Calculations**
    A critical component of the analysis is the calculation of the Electric Field Gradient (EFG) at the nucleus, as the measured energy shift is directly proportional to the product of the EFG and the quadrupole moment. For transition elements with open d-shells (like copper and cobalt), *ab initio* EFG calculations are notoriously complex and a significant source of uncertainty. The paper's reliance on accurate EFG values is highlighted by ongoing research focused on the **accuracy of EFG calculations for open d-shell atoms**. Studies investigating methods like hybrid DFT functionals to improve EFG accuracy are essential for reducing the final uncertainty in the quadrupole moments derived by Rathi et al.

### 3. Conclusion

The paper by Rathi et al. makes a significant contribution by providing new, high-precision reference values for the nuclear quadrupole moments of transition elements. The work successfully demonstrates the power of muonic atom spectroscopy as a clean and sensitive probe of nuclear structure. The findings are underpinned by the synergy between cutting-edge experimental capabilities, such as cryogenic microcalorimeter-based spectroscopy, and sophisticated, well-validated theoretical frameworks. This research not only enhances our fundamental understanding of nuclear physics but also provides benchmark data essential for refining nuclear models.

✅ Research saved to History.

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 4
Goodbye.👋


# **Summary of Changes**

***State Object:*** Instead of independent variables, we now use AgentState. This makes debugging easier (you can print the state at any point to see exactly what the agent knows).


***Nodes vs Functions:*** The logic is encapsulated in "Nodes". If you wanted to replace Tavily with Google Search, you only edit the searcher_node; the rest of the system remains untouched.

***Scalability:*** This architecture allows you to easily add loops (e.g., "If the Writer isn't happy, send back to Searcher") in future milestones.